In [1]:

import nbimporter
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
import tensorflow as tf 
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.metrics import confusion_matrix
from utils import *


In [2]:

wst_feature='../data/wstphish/western_end_features_zenodo_modify.json' 
wst_data=load_dataset(wst_feature)
# 
wst_data['label'].value_counts()
data_wst=wst_data.iloc[:,:60]#include 'label', 'index', 
data_wst=data_wst.astype(float)
data_wst.drop(['REP_SearchEngine'],axis=1,inplace=True)
 
# data_wst.columns
print('data_ch',data_wst.shape)  

1    6935
0    4269
Name: label, dtype: int64

data_ch (11204, 59)


# performance of vanilla PWD of spacephish on WstPhish, refer to Table XV,  XVI and XVII


In [96]:

# WstPhish PWD, top cell on the left side

train_wst,test_wst=train_test_split(data_wst,test_size=0.2,shuffle=True)
# print('train_ze shape',train_wst.shape)
train_wst_x=train_wst.iloc[:,0:57]
train_wst_y=train_wst.iloc[:,57] 
test_wst_x=test_wst.iloc[:,0:57] 
test_wst_y=test_wst.iloc[:,57] 

scaler_entire_wst=preprocessing.MinMaxScaler()
scaler_entire_wst.fit(train_wst_x)
train_x_wst=scaler_entire_wst.transform(train_wst_x)
test_x_wst=scaler_entire_wst.transform(test_wst_x)

#use only html features
scaler_html_wst=preprocessing.MinMaxScaler()
scaler_html_wst.fit(train_wst.iloc[:,27:49])
train_html_x_wst=scaler_html_wst.transform(train_wst.iloc[:,27:49])
test_html_x_wst=scaler_html_wst.transform(test_wst.iloc[:,27:49])
#only url features
scaler_url_wst=preprocessing.MinMaxScaler()
scaler_url_wst.fit(pd.concat([train_wst.iloc[:,0:27],train_wst.iloc[:,49:57]],axis=1))
train_url_x_wst=scaler_url_wst.transform(pd.concat([train_wst.iloc[:,0:27],train_wst.iloc[:,49:57]],axis=1)) 
test_url_x_wst=scaler_url_wst.transform(pd.concat([test_wst.iloc[:,0:27],test_wst.iloc[:,49:57]],axis=1))
test_url_y_wst=test_html_y_wst=test_wst_y
train_url_y_wst=train_html_y_wst=train_wst_y

 

pre_html_x_wst=test_wst.iloc[:,27:49]
pre_html_y_wst=pre_url_y_wst=pre_y_wst=test_wst.iloc[:,57]
pre_url_x_wst=pd.concat([test_wst.iloc[:,0:27],test_wst.iloc[:,49:57]],axis=1)
pre_x_wst=test_wst.iloc[:,0:57]
pre_x_wst=scaler_entire_wst.transform(pre_x_wst)
pre_url_x_wst=scaler_url_wst.transform(pre_url_x_wst)
pre_html_x_wst=scaler_html_wst.transform(pre_html_x_wst)

# for cnn
one_hot_train_y_wst=tf.one_hot(indices=train_wst_y.array,depth=2,on_value=1.0,off_value=0.0)
one_hot_pre_y_wst=tf.one_hot(indices=pre_y_wst.array,depth=2,on_value=1.0,off_value=0.0)
pre_x_re_wst=np.asarray(pre_x_wst).reshape(pre_x_wst.shape[0],pre_x_wst.shape[1],1)
pre_html_x_re_wst=np.asarray(pre_html_x_wst).reshape(pre_html_x_wst.shape[0],pre_html_x_wst.shape[1],1)
pre_url_x_re_wst=np.asarray(pre_url_x_wst).reshape(pre_url_x_wst.shape[0],pre_url_x_wst.shape[1],1)

train_x_re_wst=np.asarray(train_x_wst).reshape(train_x_wst.shape[0],train_x_wst.shape[1],1)

train_html_x_re_wst=np.asarray(train_html_x_wst).reshape(train_html_x_wst.shape[0],train_html_x_wst.shape[1],1)
train_url_x_re_wst=np.asarray(train_url_x_wst).reshape(train_url_x_wst.shape[0],train_url_x_wst.shape[1],1)
#whole dataset

model_rf_wst_com=RandomForestClassifier(criterion='gini', max_depth=55, min_samples_split=3, n_estimators=200,random_state=10)
model_rf_wst_url=RandomForestClassifier(criterion='gini', max_depth=50, min_samples_split=3, n_estimators=120,random_state=22)
model_rf_wst_html=RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_split=3, n_estimators=160,random_state=86)
print('combine_rf_whole:')
model_rf_com,rf_tnr_com,rf_recall_com,rf_time_com,rf_f1_com=full_metri(model_rf_wst_com,train_x_wst,train_wst_y,test_x_wst,test_wst_y)#using normalized data to train, so all the model_rf4_com data must normalized
 

print('only_url_rf_test:')
#only url
model_rf_url,rf_tnr_url,rf_recall_url,rf_time_url,rf_f1_url=full_metri(model_rf_wst_url,train_url_x_wst,train_url_y_wst,test_url_x_wst,test_url_y_wst)
#only html
print('only_html_rf_test:')
model_rf_html,rf_tnr_html,rf_recall_html,rf_time_html,rf_f1_html=full_metri(model_rf_wst_html,train_html_x_wst,train_html_y_wst,test_html_x_wst,test_html_y_wst)

print('==============lr com')
model_lr_wst_com = LogisticRegression(max_iter=500,penalty='l2',C=10,random_state=54)
model_lr_wst_url = LogisticRegression(max_iter=500,penalty='l2',C=100,random_state=65)
model_lr_wst_html = LogisticRegression(max_iter=500,penalty='l2',C=0.01,random_state=30)
print('combine lr whole:')
model_lr_com,lr_tnr_com,lr_recall_com,lr_time_com,lr_f1_com=full_metri(model_lr_wst_com,train_x_wst,train_wst_y,test_x_wst,test_wst_y)

print('only_url_lr_whole_test:')
model_lr_url,lr_tnr_url,lr_recall_url,lr_time_url,lr_f1_url=full_metri(model_lr_wst_url,train_url_x_wst,train_url_y_wst,test_url_x_wst,test_url_y_wst)

print('only_html_lr_whole_test:')
model_lr_html,lr_tnr_html,lr_recall_html,lr_time_html,lr_f1_html=full_metri(model_lr_wst_html,train_html_x_wst,train_html_y_wst,test_html_x_wst,test_html_y_wst)

 
model_cnn_com=tf.keras.models.load_model('../../wst_model_westf/model_cnn_combined_0',compile=False)

model_cnn_com.compile(loss = 'categorical_crossentropy',
      optimizer = "adam",               
            metrics = [tf.keras.metrics.CategoricalAccuracy()])

model_cnn_url=tf.keras.models.load_model('../../wst_model_westf/model_cnn_url_1',compile=False)
model_cnn_url.compile(loss = 'categorical_crossentropy', 
  optimizer ="adam", #"",#opt,adam               
            metrics = [tf.keras.metrics.CategoricalAccuracy()]) 

model_cnn_html=tf.keras.models.load_model('../../wst_model_westf/model_cnn_html_0',compile=False)
model_cnn_html.compile(loss = 'categorical_crossentropy', 
  optimizer ="adam",               
            metrics = [tf.keras.metrics.CategoricalAccuracy()]) 


print('cnn_com:')
cnn_tnr_com,cnn_recall_com,cnn_f1_com=cnn_metri(model_cnn_com,pre_x_re_wst,pre_y_wst)
print('cnn_url:')
cnn_tnr_url,cnn_recall_url,cnn_f1_url=cnn_metri(model_cnn_url,pre_url_x_re_wst,pre_y_wst)
print('cnn_html:')
cnn_tnr_html,cnn_recall_html,cnn_f1_html=cnn_metri(model_cnn_html,pre_html_x_re_wst,pre_y_wst)


train_ze shape (8963, 59)


MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

combine_rf_whole:
recall is 0.98,tnr is 0.97
only_url_rf_test:
recall is 0.98,tnr is 0.96
only_html_rf_test:
recall is 0.94,tnr is 0.91
==============lr com
combine lr whole:
recall is 0.95,tnr is 0.94
only_url_lr_whole_test:
recall is 0.95,tnr is 0.94
only_html_lr_whole_test:
recall is 0.87,tnr is 0.64
cnn_com:
recall is 1.00, tnr is 0.98
cnn_url:
recall is 0.98, tnr is 0.97
cnn_html:
recall is 0.95, tnr is 0.91


# Predict EngPhish

In [111]:

eng_feature='../data/engphish/enphish_end_features_lnu_modify.json' 
eng_data=load_dataset(eng_feature)
eng_data['label'].value_counts()
data_eng=eng_data.iloc[:,:60]#include 'label', 'index', 
data_eng=data_eng.astype(float)
data_eng.drop(['REP_SearchEngine'],axis=1,inplace=True)

# data_eng.columns
print('data_ch',data_eng.shape) 


0    11019
1     4092
Name: label, dtype: int64

data_ch (15111, 59)


In [102]:
#WstPhish PWD, top cell on the middle column
def pre_metri(model,test_x,test_y):
    pred=model.predict(test_x)
    tnr,tpr,f1_score=get_fpr(test_y,pred)
    print('recall is %.2f,tnr is %.2f'%(tpr,tnr))
    return tnr,tpr,f1_score
    

train_en,test_en=train_test_split(data_eng,test_size=0.2,shuffle=True)

pre_html_x_en=test_en.iloc[:,27:49]
pre_html_y_en=pre_url_y_en=pre_y_en=test_en.iloc[:,57]
pre_url_x_en=pd.concat([test_en.iloc[:,0:27],test_en.iloc[:,49:57]],axis=1)
pre_x_en=test_en.iloc[:,0:57]

 
pre_x_en=scaler_entire_wst.transform(pre_x_en)
pre_url_x_en=scaler_url_wst.transform(pre_url_x_en)
pre_html_x_en=scaler_html_wst.transform(pre_html_x_en)
# for cnn
one_hot_pre_y_en=tf.one_hot(indices=pre_y_en.array,depth=2,on_value=1.0,off_value=0.0)
pre_x_re_en=np.asarray(pre_x_en).reshape(pre_x_en.shape[0],pre_x_en.shape[1],1)
pre_html_x_re_en=np.asarray(pre_html_x_en).reshape(pre_html_x_en.shape[0],pre_html_x_en.shape[1],1)
pre_url_x_re_en=np.asarray(pre_url_x_en).reshape(pre_url_x_en.shape[0],pre_url_x_en.shape[1],1)

print('rf======')
rf_tnr_com_en,rf_recall_com_en,rf_f1_com=pre_metri(model_rf_com,pre_x_en,pre_y_en)
rf_tnr_url_en,rf_recall_url_en,rf_f1_url=pre_metri(model_rf_url,pre_url_x_en,pre_url_y_en)
rf_tnr_html_en,rf_recall_html_en,rf_f1_html=pre_metri(model_rf_html,pre_html_x_en,pre_html_y_en)
print('lr========')
lr_tnr_com_en,lr_recall_com_en,lr_f1_com=pre_metri(model_lr_com,pre_x_en,pre_y_en)
lr_tnr_url_en,lr_recall_url_en,lr_f1_url=pre_metri(model_lr_url,pre_url_x_en,pre_url_y_en)
lr_tnr_html_en,lr_recall_html_en,lr_f1_html=pre_metri(model_lr_html,pre_html_x_en,pre_html_y_en)
print('cnn=========')
cnn_tnr_com_en,cnn_recall_com_en,cnn_f1_com=cnn_metri(model_cnn_com,pre_x_re_en,pre_y_en)
cnn_tnr_url_en,cnn_recall_url_en,cnn_f1_url=cnn_metri(model_cnn_url,pre_url_x_re_en,pre_y_en)
cnn_tnr_html_en,cnn_recall_html_en,cnn_f1_html=cnn_metri(model_cnn_html,pre_html_x_re_en,pre_y_en)

    

rf======
recall is 0.90,tnr is 1.00
recall is 0.90,tnr is 0.99
recall is 0.67,tnr is 0.95
lr========
recall is 0.89,tnr is 1.00
recall is 0.89,tnr is 1.00
recall is 0.82,tnr is 0.73
cnn=========
recall is 0.88, tnr is 0.97
recall is 0.89, tnr is 0.99
recall is 0.70, tnr is 0.90


# Predict ChiPhish

In [108]:

chi_feature='../data/chiphish/ch_sites_total_features_update.json' 
chi_data=load_dataset(chi_feature)
 
chi_data['label'].value_counts()
data_chi=chi_data.iloc[:,:60]#  
data_chi=data_chi.astype(float)
data_chi.drop(['REP_SearchEngine'],axis=1,inplace=True)
 
print('data_ch',data_chi.shape) 

0    1055
1     565
Name: label, dtype: int64

data_ch (1620, 59)


In [107]:
#WstPhish PWD, top cell on the right column
train_ch,test_ch=train_test_split(data_chi,test_size=0.2,shuffle=True)

pre_html_x_ch=test_ch.iloc[:,27:49]
pre_html_y_ch=pre_url_y_ch=pre_y_ch=test_ch.iloc[:,57]
pre_url_x_ch=pd.concat([test_ch.iloc[:,0:27],test_ch.iloc[:,49:57]],axis=1)
pre_x_ch=test_ch.iloc[:,0:57]

pre_x_ch=scaler_entire_wst.transform(pre_x_ch)
pre_url_x_ch=scaler_url_wst.transform(pre_url_x_ch)
pre_html_x_ch=scaler_html_wst.transform(pre_html_x_ch)

# for cnn
one_hot_pre_y_ch=tf.one_hot(indices=pre_y_ch.array,depth=2,on_value=1.0,off_value=0.0)
pre_x_re_ch=np.asarray(pre_x_ch).reshape(pre_x_ch.shape[0],pre_x_ch.shape[1],1)
pre_html_x_re_ch=np.asarray(pre_html_x_ch).reshape(pre_html_x_ch.shape[0],pre_html_x_ch.shape[1],1)
pre_url_x_re_ch=np.asarray(pre_url_x_ch).reshape(pre_url_x_ch.shape[0],pre_url_x_ch.shape[1],1)
  

print('rf======')     
rf_tnr_com_ch,rf_recall_com_ch,rf_f1_com=pre_metri(model_rf_com,pre_x_ch,pre_y_ch)
rf_tnr_url_ch,rf_recall_url_ch,rf_f1_url=pre_metri(model_rf_url,pre_url_x_ch,pre_url_y_ch)
rf_tnr_html_ch,rf_recall_html_ch,rf_f1_html=pre_metri(model_rf_html,pre_html_x_ch,pre_html_y_ch)

print('lr=====')
lr_tnr_com_ch,lr_recall_com_ch,lr_f1_com=pre_metri(model_lr_com,pre_x_ch,pre_y_ch)
lr_tnr_url_ch,lr_recall_url_ch,lr_f1_url=pre_metri(model_lr_url,pre_url_x_ch,pre_url_y_ch)
lr_tnr_html_ch,lr_recall_html_ch,lr_f1_html=pre_metri(model_lr_html,pre_html_x_ch,pre_html_y_ch)

print('cnn====')
cnn_tnr_com_ch,cnn_recall_com_ch,cnn_f1_com=cnn_metri(model_cnn_com,pre_x_re_ch,pre_y_ch)
cnn_tnr_url_ch,cnn_recall_url_ch_ch,cnn_f1_url=cnn_metri(model_cnn_url,pre_url_x_re_ch,pre_y_ch)
cnn_tnr_html_ch,cnn_recall_html_ch,cnn_f1_html=cnn_metri(model_cnn_html,pre_html_x_re_ch,pre_y_ch)

rf======
recall is 0.36,tnr is 0.96
recall is 0.55,tnr is 0.92
recall is 0.28,tnr is 0.81
lr=====
recall is 0.68,tnr is 0.85
recall is 0.67,tnr is 0.79
recall is 0.65,tnr is 0.57
cnn====
recall is 0.54, tnr is 0.89
recall is 0.65, tnr is 0.76
recall is 0.27, tnr is 0.80
